In [1]:
import pandas as pd
import math
import numpy as np
import os

In [33]:
df = pd.read_csv("/home/andrewkosikowski/Documents/predictions/GeneratedMalwareAllCurrent_Final_results.csv")
df

,malware_name,ground_truth,correct_predictions,mispredictions,fireeye_102400_minus_all_0001_500_32_model,fireeye_102400_50_0001_500_32_model,fireeye_102400_minus-all-three_0001_500_32_model,malconv_102400_minus-three_0001_500_32_model,fireeye_204800_minus_one_0001_500_32_model,fireeye_204800_minus-all-three_0001_500_32_model,...,fireeye_204800_50_0001_500_32_model,fireeye_204800_minus_all_0001_500_32_model,fireeye_204800_minus-three_0001_500_32_model,malconv_102400_minus_one_0001_500_32_model,malconv_102400_minus-all-three_0001_500_32_model,malconv_204800_minus-all-three_0001_500_32_model,malconv_204800_50_0001_500_32_model,fireeye_102400_minus_one_0001_500_32_model,malconv_204800_minus-three_0001_500_32_model,malconv_102400_minus_all_0001_500_32_model
0,x86__64__lsb__unix-system-v__gcc-7.1.0__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,x86__64__lsb__unix-system-v__llvm-obfuscator-4.0.1__O0__obf-sub__unstripped__lightaidra-1.0__lightaidra,1,16,4,1,1,0,1,1,0,...,1,1,1,1,0,0,1,1,1,1
2,x86__64__lsb__unix-system-v__gcc-7.4.0__Os__no-obf__unstripped__lightaidra-1.0__lightaidra,1,16,4,1,1,0,1,1,0,...,1,1,1,1,0,0,1,1,1,1
3,x86__64__lsb__unix-system-v__gcc-6.1.0__Os__no-obf__unstripped__BASHLITE-1.0__client,1,20,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,x86__64__lsb__unix-system-v__gcc-4.9.4__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549598,x86__64__lsb__unix-system-v__clang-6.0.0__O2__no-obf__unstripped__BASHLITE-lizkebab__server_eflags_eident_91,1,20,0,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
549599,x86__64__lsb__unix-system-v__clang-4.0.0__O1__no-obf__unstripped__BASHLITE-lizkebab__client_append_215,1,18,2,1,0,0,1,1,1,...,1,1,1,1,1,1,1,1,1,1
549600,x86__64__lsb__unix-system-v__gcc-7.4.0__Os__no-obf__unstripped__Mirai-vanilla__loader_append_35,1,18,2,1,1,1,1,1,1,...,1,0,1,1,1,1,1,1,1,1
549601,x86__64__lsb__unix-system-v__gcc-8.4.0__O1__no-obf__unstripped__lightaidra-1.0__lightaidra_eflags_eident_231,1,16,4,1,1,0,1,1,0,...,1,1,1,1,0,0,1,1,1,1


In [34]:
accuracies = {}
for column in df:
    if 'model' in column:
        accuracies[column] = df[column].sum() / df.shape[0]
accuracies

{'fireeye_102400_minus_all_0001_500_32_model': 0.8113019761536964,
 'fireeye_102400_50_0001_500_32_model': 0.8719894178161327,
 'fireeye_102400_minus-all-three_0001_500_32_model': 0.6117543026511864,
 'malconv_102400_minus-three_0001_500_32_model': 0.9802985063764208,
 'fireeye_204800_minus_one_0001_500_32_model': 0.8651863254021539,
 'fireeye_204800_minus-all-three_0001_500_32_model': 0.526469105881882,
 'fireeye_102400_minus-three_0001_500_32_model': 0.863250382548858,
 'malconv_204800_minus_all_0001_500_32_model': 0.8707594390860312,
 'malconv_204800_minus_one_0001_500_32_model': 0.9732661575719201,
 'malconv_102400_50_0001_500_32_model': 0.9767450323233315,
 'fireeye_204800_50_0001_500_32_model': 0.9027607200106259,
 'fireeye_204800_minus_all_0001_500_32_model': 0.8256232225806628,
 'fireeye_204800_minus-three_0001_500_32_model': 0.8301774189733316,
 'malconv_102400_minus_one_0001_500_32_model': 0.9824746225912159,
 'malconv_102400_minus-all-three_0001_500_32_model': 0.692676350019

## Dropping the one version of models

In [35]:
df2 = df
for column in df2:
    if 'minus_one_' in column or 'minus_all_' in column:
        df2 = df2.drop(column,axis=1)
df2

,malware_name,ground_truth,correct_predictions,mispredictions,fireeye_102400_50_0001_500_32_model,fireeye_102400_minus-all-three_0001_500_32_model,malconv_102400_minus-three_0001_500_32_model,fireeye_204800_minus-all-three_0001_500_32_model,fireeye_102400_minus-three_0001_500_32_model,malconv_102400_50_0001_500_32_model,fireeye_204800_50_0001_500_32_model,fireeye_204800_minus-three_0001_500_32_model,malconv_102400_minus-all-three_0001_500_32_model,malconv_204800_minus-all-three_0001_500_32_model,malconv_204800_50_0001_500_32_model,malconv_204800_minus-three_0001_500_32_model
0,x86__64__lsb__unix-system-v__gcc-7.1.0__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
1,x86__64__lsb__unix-system-v__llvm-obfuscator-4.0.1__O0__obf-sub__unstripped__lightaidra-1.0__lightaidra,1,16,4,1,0,1,0,1,1,1,1,0,0,1,1
2,x86__64__lsb__unix-system-v__gcc-7.4.0__Os__no-obf__unstripped__lightaidra-1.0__lightaidra,1,16,4,1,0,1,0,1,1,1,1,0,0,1,1
3,x86__64__lsb__unix-system-v__gcc-6.1.0__Os__no-obf__unstripped__BASHLITE-1.0__client,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
4,x86__64__lsb__unix-system-v__gcc-4.9.4__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549598,x86__64__lsb__unix-system-v__clang-6.0.0__O2__no-obf__unstripped__BASHLITE-lizkebab__server_eflags_eident_91,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
549599,x86__64__lsb__unix-system-v__clang-4.0.0__O1__no-obf__unstripped__BASHLITE-lizkebab__client_append_215,1,18,2,0,0,1,1,1,1,1,1,1,1,1,1
549600,x86__64__lsb__unix-system-v__gcc-7.4.0__Os__no-obf__unstripped__Mirai-vanilla__loader_append_35,1,18,2,1,1,1,1,0,1,1,1,1,1,1,1
549601,x86__64__lsb__unix-system-v__gcc-8.4.0__O1__no-obf__unstripped__lightaidra-1.0__lightaidra_eflags_eident_231,1,16,4,1,0,1,0,1,1,1,1,0,0,1,1


In [36]:
accuracies2 = {}
for column in df2:
    if 'model' in column:
        accuracies2[column] = df2[column].sum() / df2.shape[0]
accuracies2

{'fireeye_102400_50_0001_500_32_model': 0.8719894178161327,
 'fireeye_102400_minus-all-three_0001_500_32_model': 0.6117543026511864,
 'malconv_102400_minus-three_0001_500_32_model': 0.9802985063764208,
 'fireeye_204800_minus-all-three_0001_500_32_model': 0.526469105881882,
 'fireeye_102400_minus-three_0001_500_32_model': 0.863250382548858,
 'malconv_102400_50_0001_500_32_model': 0.9767450323233315,
 'fireeye_204800_50_0001_500_32_model': 0.9027607200106259,
 'fireeye_204800_minus-three_0001_500_32_model': 0.8301774189733316,
 'malconv_102400_minus-all-three_0001_500_32_model': 0.6926763500199234,
 'malconv_204800_minus-all-three_0001_500_32_model': 0.6972159904512893,
 'malconv_204800_50_0001_500_32_model': 0.9756042088562108,
 'malconv_204800_minus-three_0001_500_32_model': 0.9752748802317309}

In [37]:
df2['malware_name']

0                                 x86__64__lsb__unix-system-v__gcc-7.1.0__O1__no-obf__unstripped__BASHLITE-1.0__server
1              x86__64__lsb__unix-system-v__llvm-obfuscator-4.0.1__O0__obf-sub__unstripped__lightaidra-1.0__lightaidra
2                           x86__64__lsb__unix-system-v__gcc-7.4.0__Os__no-obf__unstripped__lightaidra-1.0__lightaidra
3                                 x86__64__lsb__unix-system-v__gcc-6.1.0__Os__no-obf__unstripped__BASHLITE-1.0__client
4                                 x86__64__lsb__unix-system-v__gcc-4.9.4__O1__no-obf__unstripped__BASHLITE-1.0__server
                                                              ...                                                     
549598    x86__64__lsb__unix-system-v__clang-6.0.0__O2__no-obf__unstripped__BASHLITE-lizkebab__server_eflags_eident_91
549599          x86__64__lsb__unix-system-v__clang-4.0.0__O1__no-obf__unstripped__BASHLITE-lizkebab__client_append_215
549600                 x86__64__lsb__unix-system

In [44]:
original_malware = df2[~df2['malware_name'].str.contains('append')]
original_malware = original_malware[~original_malware['malware_name'].str.contains('eflags')]
original_malware = original_malware[~original_malware['malware_name'].str.contains('gaps')]
original_malware = original_malware[~original_malware['malware_name'].str.contains('extend_dynamic')]
original_malware = original_malware[~original_malware['malware_name'].str.contains('note_comment_debug')]
original_malware

,malware_name,ground_truth,correct_predictions,mispredictions,fireeye_102400_50_0001_500_32_model,fireeye_102400_minus-all-three_0001_500_32_model,malconv_102400_minus-three_0001_500_32_model,fireeye_204800_minus-all-three_0001_500_32_model,fireeye_102400_minus-three_0001_500_32_model,malconv_102400_50_0001_500_32_model,fireeye_204800_50_0001_500_32_model,fireeye_204800_minus-three_0001_500_32_model,malconv_102400_minus-all-three_0001_500_32_model,malconv_204800_minus-all-three_0001_500_32_model,malconv_204800_50_0001_500_32_model,malconv_204800_minus-three_0001_500_32_model
0,x86__64__lsb__unix-system-v__gcc-7.1.0__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
1,x86__64__lsb__unix-system-v__llvm-obfuscator-4.0.1__O0__obf-sub__unstripped__lightaidra-1.0__lightaidra,1,16,4,1,0,1,0,1,1,1,1,0,0,1,1
2,x86__64__lsb__unix-system-v__gcc-7.4.0__Os__no-obf__unstripped__lightaidra-1.0__lightaidra,1,16,4,1,0,1,0,1,1,1,1,0,0,1,1
3,x86__64__lsb__unix-system-v__gcc-6.1.0__Os__no-obf__unstripped__BASHLITE-1.0__client,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
4,x86__64__lsb__unix-system-v__gcc-4.9.4__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
706,x86__64__lsb__unix-system-v__gcc-4.9.0__O0__no-obf__unstripped__Mirai-vanilla__bot,1,12,8,1,0,1,0,1,1,1,1,0,0,1,1
707,x86__64__lsb__unix-system-v__clang-3.8.0__Os__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
708,x86__64__lsb__unix-system-v__clang-4.0.0__Os__no-obf__unstripped__Mirai-vanilla__loader,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
709,x86__64__lsb__unix-system-v__gcc-5.1.0__O3__no-obf__unstripped__BASHLITE-lizkebab__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1


In [54]:
original_passed = df2
to_drop = []
mispredicted = original_malware[original_malware['mispredictions'] != 0]

for index,row in mispredicted.iterrows():
    to_drop.append(row['malware_name'])
to_drop

['x86__64__lsb__unix-system-v__llvm-obfuscator-4.0.1__O0__obf-sub__unstripped__lightaidra-1.0__lightaidra',
 'x86__64__lsb__unix-system-v__gcc-7.4.0__Os__no-obf__unstripped__lightaidra-1.0__lightaidra',
 'x86__64__lsb__unix-system-v__clang-5.0.1__O0__no-obf__unstripped__Mirai-vanilla__loader',
 'x86__64__lsb__unix-system-v__gcc-6.5.0__O0__no-obf__unstripped__lightaidra-1.0__lightaidra',
 'x86__64__lsb__unix-system-v__clang-4.0.0__O2__no-obf__unstripped__Mirai-vanilla__loader',
 'x86__64__lsb__unix-system-v__clang-3.9.0__O0__no-obf__unstripped__Mirai-vanilla__loader',
 'x86__64__lsb__unix-system-v__gcc-5.5.0__O3__no-obf__unstripped__Mirai-vanilla__bot',
 'x86__64__lsb__unix-system-v__clang-3.8.0__O0__no-obf__unstripped__BASHLITE-lizkebab__server',
 'x86__64__lsb__unix-system-v__gcc-5.1.0__O1__no-obf__unstripped__Mirai-vanilla__loader',
 'x86__64__lsb__unix-system-v__gcc-8.1.0__O0__no-obf__unstripped__Mirai-vanilla__bot',
 'x86__64__lsb__unix-system-v__gcc-7.4.0__O1__no-obf__unstripped__

In [55]:
for name in to_drop:
    original_passed = original_passed[~original_passed['malware_name'].str.contains(name)]

In [59]:
original_passed

,malware_name,ground_truth,correct_predictions,mispredictions,fireeye_102400_50_0001_500_32_model,fireeye_102400_minus-all-three_0001_500_32_model,malconv_102400_minus-three_0001_500_32_model,fireeye_204800_minus-all-three_0001_500_32_model,fireeye_102400_minus-three_0001_500_32_model,malconv_102400_50_0001_500_32_model,fireeye_204800_50_0001_500_32_model,fireeye_204800_minus-three_0001_500_32_model,malconv_102400_minus-all-three_0001_500_32_model,malconv_204800_minus-all-three_0001_500_32_model,malconv_204800_50_0001_500_32_model,malconv_204800_minus-three_0001_500_32_model
0,x86__64__lsb__unix-system-v__gcc-7.1.0__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
3,x86__64__lsb__unix-system-v__gcc-6.1.0__Os__no-obf__unstripped__BASHLITE-1.0__client,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
4,x86__64__lsb__unix-system-v__gcc-4.9.4__O1__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
8,x86__64__lsb__unix-system-v__clang-4.0.0__O2__no-obf__unstripped__BASHLITE-1.0__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
13,x86__64__lsb__unix-system-v__gcc-10.1.0__O1__no-obf__unstripped__BASHLITE-lizkebab__server,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
549591,x86__64__lsb__unix-system-v__clang-6.0.0__O2__no-obf__unstripped__BASHLITE-1.0__server_append_137,1,15,5,0,1,1,1,1,1,1,0,1,1,1,1
549595,x86__64__lsb__unix-system-v__clang-3.9.0__O2__no-obf__unstripped__BASHLITE-1.0__server_gaps_87,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
549596,x86__64__lsb__unix-system-v__llvm-obfuscator-4.0.1__O0__obf-bcf__unstripped__BASHLITE-lizkebab__server_eflags_eident_197,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1
549598,x86__64__lsb__unix-system-v__clang-6.0.0__O2__no-obf__unstripped__BASHLITE-lizkebab__server_eflags_eident_91,1,20,0,1,1,1,1,1,1,1,1,1,1,1,1


In [62]:
original_passed.sum(axis=0,numeric_only=True) / original_passed.shape[0]

ground_truth                                         1.000000
correct_predictions                                 18.719292
mispredictions                                       1.280708
fireeye_102400_50_0001_500_32_model                  0.886334
fireeye_102400_minus-all-three_0001_500_32_model     0.873793
malconv_102400_minus-three_0001_500_32_model         0.991261
fireeye_204800_minus-all-three_0001_500_32_model     0.816052
fireeye_102400_minus-three_0001_500_32_model         0.887804
malconv_102400_50_0001_500_32_model                  0.995080
fireeye_204800_50_0001_500_32_model                  0.907801
fireeye_204800_minus-three_0001_500_32_model         0.856975
malconv_102400_minus-all-three_0001_500_32_model     0.995325
malconv_204800_minus-all-three_0001_500_32_model     0.990167
malconv_204800_50_0001_500_32_model                  0.990046
malconv_204800_minus-three_0001_500_32_model         0.984878
dtype: float64

In [65]:
original_malware_correctly_predicted = original_malware.drop(mispredicted.index)

In [67]:
original_malware.sum(axis=0,numeric_only=True) / original_malware.shape[0]

ground_truth                                         1.000000
correct_predictions                                 17.998594
mispredictions                                       2.001406
fireeye_102400_50_0001_500_32_model                  0.992968
fireeye_102400_minus-all-three_0001_500_32_model     0.676512
malconv_102400_minus-three_0001_500_32_model         0.990155
fireeye_204800_minus-all-three_0001_500_32_model     0.592124
fireeye_102400_minus-three_0001_500_32_model         0.983122
malconv_102400_50_0001_500_32_model                  0.980309
fireeye_204800_50_0001_500_32_model                  0.990155
fireeye_204800_minus-three_0001_500_32_model         0.988748
malconv_102400_minus-all-three_0001_500_32_model     0.693390
malconv_204800_minus-all-three_0001_500_32_model     0.700422
malconv_204800_50_0001_500_32_model                  0.987342
malconv_204800_minus-three_0001_500_32_model         0.991561
dtype: float64

In [68]:
df2.sum(axis=0,numeric_only=True) / df2.shape[0]

ground_truth                                         1.000000
correct_predictions                                 17.000702
mispredictions                                       2.999298
fireeye_102400_50_0001_500_32_model                  0.871989
fireeye_102400_minus-all-three_0001_500_32_model     0.611754
malconv_102400_minus-three_0001_500_32_model         0.980299
fireeye_204800_minus-all-three_0001_500_32_model     0.526469
fireeye_102400_minus-three_0001_500_32_model         0.863250
malconv_102400_50_0001_500_32_model                  0.976745
fireeye_204800_50_0001_500_32_model                  0.902761
fireeye_204800_minus-three_0001_500_32_model         0.830177
malconv_102400_minus-all-three_0001_500_32_model     0.692676
malconv_204800_minus-all-three_0001_500_32_model     0.697216
malconv_204800_50_0001_500_32_model                  0.975604
malconv_204800_minus-three_0001_500_32_model         0.975275
dtype: float64